# import

In [17]:
import pandas as pd
import numpy as np

# pyod
from pyod.models.knn import KNN
from pyod.utils.data import generate_data

# sklearn
import sklearn
from sklearn import metrics
from sklearn.metrics import f1_score

metrics = [sklearn.metrics.accuracy_score,
           sklearn.metrics.precision_score,
           sklearn.metrics.recall_score,
           sklearn.metrics.f1_score]

# gcn
import torch
import torch.nn.functional as F
import torch_geometric
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data, DataLoader

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(data.num_node_features, 16)
        self.conv2 = GCNConv(16,2)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

# pyod

In [2]:
np.random.seed(42)
X, XX, y, yy = generate_data(n_train=200, n_test=100, contamination=0.1)
clf = KNN()
clf.fit(X)
yyhat = clf.predict(XX)
_results1= pd.DataFrame({m.__name__:[m(yy,yyhat).round(6)] for m in metrics},index=['분석1'])
_results1

,accuracy_score,precision_score,recall_score,f1_score
분석1,0.98,0.833333,1.0,0.909091


# GCN

In [22]:
np.random.seed(42)
X, XX, y, yy = generate_data(n_train=200, n_test=100, contamination=0.1)
X_combined = np.vstack((X, XX))
y_combined = np.hstack((y, yy))

num_nodes = data.num_nodes
edge_index = torch.tensor([(i, j) for i in range(num_nodes) for j in range(num_nodes) if i != j], dtype=torch.long).t()
data = Data(x=torch.tensor(X_combined, dtype=torch.float32), y=torch.tensor(y_combined, dtype=torch.long), edge_index=edge_index)

model = GCN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.NLLLoss()

train_loader = DataLoader([data], batch_size=1, shuffle=True)

model.train()
for epoch in range(100):
    for batch in train_loader:
        optimizer.zero_grad()
        out = model(batch)
        loss = criterion(out, batch.y)
        loss.backward()
        optimizer.step()

model.eval() 
with torch.no_grad():
    out = model(data)
    yyhat = out.argmax(dim=1)


accuracy = accuracy_score(y_combined, predicted)
f1 = f1_score(y_combined, predicted, average='micro')
precision = precision_score(y_combined, predicted, average='micro')
recall = recall_score(y_combined, predicted, average='micro')


print("Accuracy:", accuracy)
print("F1 Score:", f1)
print("Precision:", precision)
print("Recall:", recall)
    

# _results2= pd.DataFrame({m.__name__:[m(yy,yyhat).round(6)] for m in metrics},index=['분석2'])
# _results2


/home/coco/anaconda3/envs/py38/lib/python3.8/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Accuracy: 0.9
F1 Score: 0.9
Precision: 0.9
Recall: 0.9
